# 🛠️ Olist Feature Engineering Pipeline

## 🎯 Objective
Transform raw Olist tables into a single **Master Table** for Machine Learning. 
Key focus: **Geospatial Engineering** (Seller-Customer Distance) and **Time-based Features**.

In [1]:
import pandas as pd
import numpy as np
import os

DATA_DIR = "olist_data"

def load_csv(name):
    return pd.read_csv(os.path.join(DATA_DIR, f"olist_{name}_dataset.csv"))

print("Loading datasets...")
orders = load_csv("orders")
items = load_csv("order_items")
sub_orders = orders[orders['order_status'] == 'delivered'].copy()
customers = load_csv("customers")
sellers = load_csv("sellers")
products = load_csv("products")
geo = load_csv("geolocation")

print(f"Delivered Orders: {sub_orders.shape}")

Loading datasets...
Delivered Orders: (96478, 8)


## 1. Geospatial Data Cleaning
The `geolocation` dataset has duplicates (multiple lat/lng entries for the same zip code). We will aggregate by zip code using the **median** latitude and longitude to get a single centroid.

In [2]:
# Group by Zip Code and take Median (Robust to outliers)
geo_agg = geo.groupby('geolocation_zip_code_prefix')[['geolocation_lat', 'geolocation_lng']].median().reset_index()
print(f"Unique Zip Codes: {geo_agg.shape[0]}")
geo_agg.head(3)

Unique Zip Codes: 19015


,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng
0,1001,-23.550381,-46.634027
1,1002,-23.548551,-46.635072
2,1003,-23.548977,-46.635313


## 2. Master Table Merging
Strategy: `Orders` -> `Items` -> `Products` -> `Sellers` -> `Customers`.

In [3]:
# 1. Orders + Items
master = sub_orders.merge(items, on='order_id', how='left')

# 2. + Products (Dimensions/Weight)
master = master.merge(products, on='product_id', how='left')

# 3. + Sellers (Zip Code)
master = master.merge(sellers, on='seller_id', how='left')

# 4. + Customers (Zip Code)
master = master.merge(customers, on='customer_id', how='left')

print(f"Master shape after basic merges: {master.shape}")

Master shape after basic merges: (110197, 29)


## 3. Adding Coordinates (Lat/Lng)

In [4]:
# Merge Seller Coordinates
master = master.merge(
    geo_agg, 
    left_on='seller_zip_code_prefix', 
    right_on='geolocation_zip_code_prefix', 
    how='left'
).rename(columns={'geolocation_lat': 'seller_lat', 'geolocation_lng': 'seller_lng'})

# Drop redundant join key
master.drop(columns='geolocation_zip_code_prefix', inplace=True)

# Merge Customer Coordinates
master = master.merge(
    geo_agg, 
    left_on='customer_zip_code_prefix', 
    right_on='geolocation_zip_code_prefix', 
    how='left'
).rename(columns={'geolocation_lat': 'customer_lat', 'geolocation_lng': 'customer_lng'})

master.drop(columns='geolocation_zip_code_prefix', inplace=True)

# Check for missing coords
missing_seller = master['seller_lat'].isnull().sum()
missing_cust = master['customer_lat'].isnull().sum()
print(f"Rows missing coords: Seller ({missing_seller}), Customer ({missing_cust})")

# Drop rows where we can't calculate distance (fundamental for this project)
master.dropna(subset=['seller_lat', 'seller_lng', 'customer_lat', 'customer_lng'], inplace=True)
print(f"Cleaned Master Shape: {master.shape}")

Rows missing coords: Seller (249), Customer (288)
Cleaned Master Shape: (109661, 33)


## 4. Feature: Haversine Distance
Calculate the great-circle distance between two points on a sphere given their longitudes and latitudes.

In [5]:
def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # Convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    # Haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

master['distance_km'] = haversine_np(
    master['seller_lng'], master['seller_lat'],
    master['customer_lng'], master['customer_lat']
)

print(f"Avg Distance: {master['distance_km'].mean():.2f} km")

Avg Distance: 596.18 km


## 5. Target Variable & Time Features

In [6]:
# Convert Timestamps
time_cols = ['order_purchase_timestamp', 'order_delivered_customer_date', 'order_estimated_delivery_date']
for col in time_cols:
    master[col] = pd.to_datetime(master[col])

# Target: Actual Delivery Days
master['actual_delivery_days'] = (master['order_delivered_customer_date'] - master['order_purchase_timestamp']).dt.total_seconds() / 86400

# Cleaning: Drop negative delivery times (data errors) or absurdly long ones (>100 days?)
master = master[master['actual_delivery_days'] > 0]
master = master[master['actual_delivery_days'] < 60]  # Filter extreme outliers for better training stability

# Features
master['estimated_delivery_days'] = (master['order_estimated_delivery_date'] - master['order_purchase_timestamp']).dt.total_seconds() / 86400
master['purchase_year'] = master['order_purchase_timestamp'].dt.year
master['purchase_month'] = master['order_purchase_timestamp'].dt.month
master['purchase_weekday'] = master['order_purchase_timestamp'].dt.weekday
master['is_same_state'] = (master['seller_state'] == master['customer_state']).astype(int)

print("Added target and time features.")

Added target and time features.


## 6. Save Processed Data
We will save this as `olist_processed.csv` for the modeling phase.

In [7]:
# Select relevant columns to keep filesize manageable
cols_to_keep = [
    'order_id', 'product_id', 'seller_id', 
    'price', 'freight_value', 'product_weight_g', 'product_volume_cm3', # Note: Need to calc volume if not exists
    'distance_km', 'is_same_state',
    'purchase_year', 'purchase_month', 'purchase_weekday',
    'estimated_delivery_days', 'actual_delivery_days',  # TARGET
    'order_purchase_timestamp' # Kept for Rolling Split
]

# Calculate Volume (L * H * W) if missing
master['product_volume_cm3'] = master['product_length_cm'] * master['product_height_cm'] * master['product_width_cm']
master.dropna(subset=['product_weight_g', 'product_volume_cm3'], inplace=True)

final_df = master[cols_to_keep].copy()
final_df.sort_values('order_purchase_timestamp', inplace=True)

final_df.to_csv("olist_processed.csv", index=False)
print(f"✅ Saved processed dataset with shape: {final_df.shape}")
print("Sample:")
final_df.head()

✅ Saved processed dataset with shape: (109305, 15)
Sample:


,order_id,product_id,seller_id,price,freight_value,product_weight_g,product_volume_cm3,distance_km,is_same_state,purchase_year,purchase_month,purchase_weekday,estimated_delivery_days,actual_delivery_days,order_purchase_timestamp
34057,bfbd0f9bdef84302105ad712db648a6c,5a6b04657a4c5ee34285d1e4619a96b4,ecccfa2bb93b34a3bf033cc5d1dcdc69,44.99,2.83,1000.0,4096.0,566.040211,0,2016,9,3,18.488449,54.813194,2016-09-15 12:16:38
34058,bfbd0f9bdef84302105ad712db648a6c,5a6b04657a4c5ee34285d1e4619a96b4,ecccfa2bb93b34a3bf033cc5d1dcdc69,44.99,2.83,1000.0,4096.0,566.040211,0,2016,9,3,18.488449,54.813194,2016-09-15 12:16:38
34056,bfbd0f9bdef84302105ad712db648a6c,5a6b04657a4c5ee34285d1e4619a96b4,ecccfa2bb93b34a3bf033cc5d1dcdc69,44.99,2.83,1000.0,4096.0,566.040211,0,2016,9,3,18.488449,54.813194,2016-09-15 12:16:38
103330,3b697a20d9e427646d92567910af6d57,3ae08df6bcbfe23586dd431c40bddbb7,522620dcb18a6b31cd7bdf73665113a9,29.90,15.56,300.0,4096.0,708.535291,0,2016,10,0,23.593866,23.178738,2016-10-03 09:44:50
31500,be5bc2f0da14d8071e2d45451ad119d9,fd7fd78fd3cbc1b0a6370a7909c0a629,f09b760d23495ac9a7e00d29b769007c,21.90,17.19,400.0,4096.0,915.734331,0,2016,10,0,34.293866,24.057500,2016-10-03 16:56:50
